##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Salvar e Carregar Modelos

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/save_and_load"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Ver em TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/save_and_load.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Executar no Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/keras/save_and_load.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Ver código fonte no GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/keras/save_and_load.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Baixar notebook</a>
  </td>
</table>

Para recapitular: aqui estão as maneiras mais comuns de evitar o ajuste excessivo nas redes neurais:

* Obtenha mais dados de treinamento.
* Reduza a capacidade da rede.
* Adicione regularização de peso.
* Adicione desistência.

O progresso do modelo pode ser salvo durante e após o treinamento. Isso significa que um modelo pode retomar de onde parou e evitar longos períodos de treinamento. Salvar também significa que você pode compartilhar seu modelo e outras pessoas podem recriar seu trabalho. Ao publicar modelos e técnicas de pesquisa, a maioria dos profissionais de aprendizado de máquina compartilha:

* código para criar o modelo e
* os pesos ou parâmetros treinados para o modelo

O compartilhamento desses dados ajuda outras pessoas a entender como o modelo funciona e a experimentar com novos dados.

Cuidado: Tenha cuidado com o código não confiável - os modelos TensorFlow são código. Consulte [Usando o TensorFlow com segurança] (https://github.com/tensorflow/tensorflow/blob/master/SECURITY.md) para obter detalhes.

### Opções

Existem diferentes maneiras de salvar os modelos do TensorFlow, dependendo da API que você está usando. Este guia usa [tf.keras] (https://www.tensorflow.org/guide/keras), uma API de alto nível para criar e treinar modelos no TensorFlow. Para outras abordagens, consulte o guia TensorFlow [Save and Restore] (https://www.tensorflow.org/guide/saved_model) ou [Saving in eager](https://www.tensorflow.org/guide/eager#object-based_saving).

## Configuração

### Instalações e importações

Instale e importe o TensorFlow e as dependências:

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

!pip install pyyaml h5py  # Necessário para salvar modelos em formato HDF5

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

### Obtenha um exemplo de conjunto de dados

Para demonstrar como salvar e carregar pesos, você usará o [MNIST dataset] (http://yann.lecun.com/exdb/mnist/). Para acelerar essas execuções, use os primeiros 1000 exemplos:

In [0]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

### Definir um modelo

Comece criando um modelo seqüencial simples:

In [0]:
# Definindo um modelo sequencial simples
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  return model

# Crie uma instância básica do modelo
model = create_model()

# Exibir a arquitetura do modelo
model.summary()

## Salvar checkpoints durante o treinamento

Você pode usar um modelo treinado sem precisar treiná-lo novamente ou pegar o treinamento de onde parou - caso o processo de treinamento tenha sido interrompido. O retorno de chamada `tf.keras.callbacks.ModelCheckpoint` permite salvar continuamente o modelo *durante* e *no final* do treinamento.

### Uso de retorno de chamada no ponto de verificação

Crie um retorno de chamada `tf.keras.callbacks.ModelCheckpoint` que economize pesos apenas durante o treinamento:

In [0]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Crie um retorno de chamada que salve os pesos do modelo
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Treine o modelo com o novo retorno de chamada
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# Isso pode gerar avisos relacionados ao salvamento do estado do otimizador.
# Esses avisos (e avisos semelhantes em todo este notebook)
# existem para desencorajar o uso desatualizado e podem ser ignorados.

Isso cria uma única coleção de arquivos de checkpoints do TensorFlow que são atualizados no final de cada época:

In [0]:
!ls {checkpoint_dir}

Crie um novo modelo não treinado. Ao restaurar um modelo somente com pesos, você deve ter um modelo com a mesma arquitetura que o modelo original. Como é a mesma arquitetura de modelo, você pode compartilhar pesos, apesar de ser uma *instância* diferente do modelo.

Agora reconstrua um modelo novo e não treinado e avalie-o no conjunto de testes. Um modelo não treinado terá desempenho em níveis de chance (~ 10% de precisão):

In [0]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

Em seguida, carregue os pesos no ponto de verificação e reavalie:

In [0]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

### Opções de retorno de chamada de checkpoints

O retorno de chamada fornece várias opções para fornecer nomes exclusivos para checkpoints e ajustar a frequência do checkpoint.

Treine um novo modelo e salve checkpoints com nomes exclusivos uma vez a cada cinco épocas:

In [0]:
# Inclua a época no nome do arquivo (usa `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Crie um retorno de chamada que salve os pesos do modelo a cada 5 épocas
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)

# Crie uma nova instância de modelo
model = create_model()

# Salve os pesos usando o formato `checkpoint_path`
model.save_weights(checkpoint_path.format(epoch=0))

# Treine o modelo com o novo retorno de chamada
model.fit(train_images, 
              train_labels,
              epochs=50, 
              callbacks=[cp_callback],
              validation_data=(test_images,test_labels),
              verbose=0)

Agora, observe os pontos de verificação resultantes e escolha o mais recente:

In [0]:
!ls {checkpoint_dir}

In [0]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

Nota: o formato padrão do tensorflow salva apenas os 5 checkpoints mais recentes.

Para testar, redefina o modelo e carregue o checkpoint mais recente:

In [0]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

## Quais são esses arquivos?

O código acima armazena os pesos em uma coleção de arquivos formatados [https://www.tensorflow.org/guide/saved_model#save_and_restore_variables) que contêm apenas os pesos treinados em um formato binário. Os pontos de verificação contêm:
* Um ou mais fragmentos que contêm os pesos do seu modelo.
* Um arquivo de índice que indica quais pesos estão armazenados em um shard.

Se você estiver treinando apenas um modelo em uma única máquina, terá um fragmento com o sufixo: `.data-00000-of-00001`

## Salvar manualmente pesos

Você viu como carregar os pesos em um modelo. Salvar manualmente é tão simples quanto o método `Model.save_weights`. Por padrão, `tf.keras` - e `save_weights` em particular - usa o formato TensorFlow [Checkpoint] (../../guide/checkpoint.ipynb) com uma extensão `.ckpt` (salvando em [HDF5]) (https://js.tensorflow.org/tutorials/import-keras.html) com uma extensão `.h5` é abordada em [Salvar e serializar modelos] (../../guide/keras/save_and_serialize#weights-only_saving_in_savedmodel_format):

In [0]:
# Salvar pesos
model.save_weights('./checkpoints/my_checkpoint')

# Criar nova instancia do modelo
model = create_model()

# Restaurar os pesos
model.load_weights('./checkpoints/my_checkpoint')

# Avaliar o modelo
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

## Salve o modelo inteiro

Ligue para [`model.save`] (https://www.tensorflow.org/api_docs/python/tf/keras/Model#save) para salvar a arquitetura, pesos e configuração de treinamento de um modelo em um único arquivo/pasta. Isso permite exportar um modelo para que ele possa ser usado sem acesso ao código* Python original. Como o estado do otimizador é recuperado, você pode retomar o treinamento exatamente de onde parou.

Salvar um modelo totalmente funcional é muito útil - você pode carregá-lo no TensorFlow.js ([HDF5] (https://js.tensorflow.org/tutorials/import-keras.html), [Modelo salvo] (https: / /js.tensorflow.org/tutorials/import-saved-model.html)) e depois treiná-los e executá-los em navegadores da Web ou convertê-los para rodar em dispositivos móveis usando o TensorFlow Lite ([HDF5] (https: // www. tensorflow.org/lite/convert/python_api#exporting_a_tfkeras_file_), [Modelo salvo] (https://www.tensorflow.org/lite/convert/python_api#exporting_a_savedmodel_))

*Objetos personalizados (por exemplo, modelos ou camadas subclasses) requerem atenção especial ao salvar e carregar. Consulte a seção **Salvando objetos personalizados** abaixo

### Formato HDF5

O Keras fornece um formato básico de salvamento usando o padrão [HDF5] (https://en.wikipedia.org/wiki/Hierarchical_Data_Format).

In [0]:
# Criar e treinar uma nova instancia de modelo
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Salvar o modelo inteiro.
# A extensão '.h5' indica que o modelo será salvo em HDF5.
model.save('my_model.h5') 

Agora, recrie o modelo a partir do arquivo:

In [0]:
# Recriar o modelo incluindo seus pesos e otimizador
new_model = tf.keras.models.load_model('my_model.h5')

# Mostrar a arquitetura do modelo
new_model.summary()

Verificar a acurácia:

In [0]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

Essa técnica salva tudo:

* Os valores de peso
* A configuração do modelo (arquitetura)
* A configuração do otimizador

Keras salva modelos inspecionando a arquitetura. Atualmente, não é possível salvar os otimizadores do TensorFlow (do `tf.train`). Ao usá-los, você precisará recompilar o modelo após o carregamento e perderá o estado do otimizador.


### Formato SavedModel

O formato SavedModel é outra maneira de serializar modelos. Os modelos salvos neste formato podem ser restaurados usando o `tf.keras.models.load_model` e são compatíveis com o TensorFlow Serving. O [guia SavedModel] (https://www.tensorflow.org/guide/saved_model) detalha como servir/inspecionar o SavedModel. A seção abaixo ilustra as etapas para salvar e restaurar o modelo.

In [0]:
# Criar e treinar uma nova instância do modelo.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Salvar o modelo inteiro com SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model') 

O formato SavedModel é um diretório que contém um binário protobuf e um checkpoint Tensorflow. Inspecione o diretório do modelo salvo:

In [0]:
# diretório my_model
!ls saved_model

# Contém uma pasta de ativos, saved_model.pb e pasta de variáveis.
!ls saved_model/my_model

Recarregue um novo modelo Keras a partir do modelo salvo:

In [0]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Verifique a arquitetura
new_model.summary()

O modelo restaurado é compilado com os mesmos argumentos que o modelo original. Tente executar avaliar e prever com o modelo carregado:

In [0]:
# Avaliando o modelo restaurado
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

### Salvando objetos personalizados

Se você estiver usando o formato SavedModel, poderá pular esta seção. A principal diferença entre HDF5 e SavedModel é que o HDF5 usa configurações de objetos para salvar a arquitetura do modelo, enquanto SavedModel salva o grafo de execução. Assim, SavedModels podem salvar objetos personalizados, como modelos subclassificados e camadas personalizadas, sem exigir o código original.

Para salvar objetos personalizados no HDF5, faça o seguinte:

1. Defina um método `get_config` no seu objeto e, opcionalmente, um método de classe `from_config`.
  * `get_config (self)` retorna um dicionário serializável em JSON de parâmetros necessários para recriar o objeto.
  * `from_config (cls, config)` usa a configuração retornada de `get_config` para criar um novo objeto. Por padrão, esta função usará a configuração como kwargs de inicialização (`return cls (** config)`).
2. Passe o objeto para o argumento `custom_objects` ao carregar o modelo. O argumento deve ser um dicionário que mapeie o nome da classe de string para a classe Python. Por exemplo. `tf.keras.models.load_model (caminho, custom_objects = {'CustomLayer': CustomLayer})`

Veja o tutorial [Escrevendo camadas e modelos do zero] (https://www.tensorflow.org/guide/keras/custom_layers_and_models) para obter exemplos de objetos personalizados e `get_config`.
